In [ ]:
# Import the necessary libraries 
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
from PIL import Image

In [ ]:
# Check if a GPU is available, if so, use it; otherwise, use CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Mount Google Drive to access data
from google.colab import drive
drive.mount('/content/drive/')


In [ ]:
# Read training and testing data from CSV files
read_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/kaggleData/train_data.csv', delimiter=',', skiprows=0)
read_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/kaggleData/test_data.csv', delimiter=',', skiprows=0, header=None)
read_target = pd.read_csv('/content/drive/My Drive/Colab Notebooks/kaggleData/train_target.csv', delimiter=',', skiprows=0)

# Extract target values
targetValues = read_target.iloc[:, 0].values

# Define a custom transformation to convert data to PIL format
class ToPILImage(object):
    def __call__(self, tensor):
        return transforms.ToPILImage()(tensor)

# Building Tranformations
transform = transforms.Compose([
    ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.GaussianBlur(3),
    transforms.ToTensor()
])

#Convert to numpy
train_numpy = read_train.to_numpy()
test_numpy = read_test.to_numpy()
target_numpy = read_target.to_numpy()

#Normalizing
train_numpy = train_numpy / 225.0
test_numpy = test_numpy / 225.0

#Converting into tensor
train_tensor = torch.tensor(train_numpy, dtype=torch.float32)
target_tensor = torch.tensor(targetValues)
test_tensor = torch.tensor(test_numpy, dtype=torch.float32)

#Reshape the input data from 3d into 4D
train_tensor = train_tensor.view(-1, 1, 48, 48)
test_tensor = test_tensor.view(-1, 1, 48, 48)

#Transform dataset
dataset = TensorDataset(train_tensor, target_tensor)
dataset_transformed = [(transform(x), y) for x, y in dataset]

target_dataset = TensorDataset(target_tensor)

# Create DataLoader for train data
kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available()else {}
train_dataloader = DataLoader(dataset_transformed, batch_size=16, shuffle=True, **kwargs)



In [ ]:
import torch.nn as nn

# Define a neural network model with batch normalization
class NetBatchNorm(nn.Module):
    def __init__(self):
        super(NetBatchNorm, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.fc1 = nn.Linear(256 * 3 * 3, 512)
        self.drop1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.drop2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, 7)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.bn1(x)
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv2(x))
        x = self.bn2(x)
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv3(x))
        x = self.bn3(x)
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv4(x))
        x = self.bn4(x)
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.drop1(x)
        x = torch.relu(self.fc2(x))
        x = self.drop2(x)
        x = self.fc3(x)
        return x

model = NetBatchNorm()

In [ ]:
learning_rate = 0.001
model = model.to(device)

# Move the model's weight tensors to the same device as the model (GPU)
model = model.to(device)
#optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the loss function
loss_fn = nn.CrossEntropyLoss()

n_epochs = 15

for epoch in range(n_epochs):
    model.train(True)
    for imgs,labels in train_dataloader:
        imgs, labels = imgs.to(device), labels.to(device)

        outputs = model(imgs)

        train_loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

    model.eval()
    print("Epoch: %d, train_loss: %f" % (epoch, float(train_loss)))

In [ ]:
test_tensor = test_tensor.to(device)
with torch.no_grad():
    test_predictions = model(test_tensor)

# Get the predicted class labels
    test_predictions = torch.max(test_predictions, dim=1)

In [ ]:
submission_df = pd.DataFrame({
    "Id": range(len(test_predictions)),
    "Category": test_predictions.cpu().numpy()
})

# Save the DataFrame to a CSV file
submission_df.to_csv('/content/drive/My Drive/Colab Notebooks/kaggleData/submission.csv', index=False)